In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

Preprocesamiento

In [3]:
tr_df = pd.read_csv("Train_data.csv")
tst_df = pd.read_csv("Test_data.csv")

In [4]:
print('Number of columns:', len(tr_df.columns))
tr_df.head()

Number of columns: 42


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [13]:
def checkAttack(value):
  if value == 'normal': return 0
  return 1
numeric_data = tr_df.select_dtypes(include=np.number)
numeric_data["isAttack"] = tr_df["class"].apply(checkAttack)
numeric_data.astype(np.number)
numeric_data.head()


def check_attack(value):
    """
    Retorna 0 si el valor es "normal", 1 en cualquier otro caso.
    """
    return 0 if value == 'normal' else 1

# Selecciona solo las columnas numéricas del DataFrame `tr_df`
numeric_data = tr_df.select_dtypes(include=np.number)

# Agrega una nueva columna "isAttack" al DataFrame `numeric_data`
numeric_data["isAttack"] = tr_df["class"].apply(check_attack)

# Convierte todas las columnas del DataFrame `numeric_data` a tipo de datos numérico
numeric_data = numeric_data.astype(np.number)

# Muestra las primeras filas del DataFrame resultante
print(numeric_data.head())




   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0       0.0      491.0        0.0   0.0             0.0     0.0  0.0   
1       0.0      146.0        0.0   0.0             0.0     0.0  0.0   
2       0.0        0.0        0.0   0.0             0.0     0.0  0.0   
3       0.0      232.0     8153.0   0.0             0.0     0.0  0.0   
4       0.0      199.0      420.0   0.0             0.0     0.0  0.0   

   num_failed_logins  logged_in  num_compromised  ...  dst_host_srv_count  \
0                0.0        0.0              0.0  ...                25.0   
1                0.0        0.0              0.0  ...                 1.0   
2                0.0        0.0              0.0  ...                26.0   
3                0.0        1.0              0.0  ...               255.0   
4                0.0        1.0              0.0  ...               255.0   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.17                    0.03  

In [16]:
for column in tr_df.columns:
  if column in numeric_data.columns: continue

  print('\nColumn:', column)
  print(tr_df[column].unique())
numeric_data.describe()

numeric_data['isAttack'].value_counts()
X = numeric_data.drop(['isAttack'], axis=1)
y = numeric_data['isAttack']


Column: protocol_type
['tcp' 'udp' 'icmp']

Column: service
['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001']

Column: flag
['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH']

Column: class
['normal' 'anomaly']


In [39]:
def generate_model(data, y):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.30, random_state=42)

    # Crear un clasificador KNN y entrenarlo en los datos de entrenamiento
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)

    # Predecir la variable objetivo en los datos de prueba utilizando el modelo entrenado
    y_pred = knn.predict(X_test)

    # Calcular la matriz de confusión
    result_arbol = confusion_matrix(y_test, y_pred)
    print("TP: ", result_arbol[0][0])
    print("FP: ", result_arbol[0][1])
    print("FN: ", result_arbol[1][0])
    print("TN: ", result_arbol[1][1], '\n')

    print(result_arbol)

    # Calcular las métricas de evaluación del modelo y mostrarlas en pantalla
    print ('\nAccuracy:', accuracy_score(y_test, y_pred))
    print ("Precision:", precision_score(y_test, y_pred))
    print ("Recall: ", recall_score(y_test, y_pred))
    print ("F1: ", f1_score(y_test, y_pred))

    return knn

In [44]:
def write_csv(y_preds, filename):
    # Convertir las predicciones en un array numpy y crear un DataFrame con él
  array = np.asarray([y_preds])

    # Escribir el DataFrame en un archivo CSV
  pd.DataFrame(array).to_csv(filename, header=False)
 

In [45]:
 test_numeric_data = tst_df.select_dtypes(include=np.number)

In [46]:
model = generateModel(X)

TP:  3988
FP:  54
FN:  59
TN:  3457 

[[3988   54]
 [  59 3457]]

Accuracy: 0.9850489547499338
Precision: 0.9846197664483053
Recall:  0.9832195676905574
F1:  0.9839191689198804


In [47]:
y_pred = model.predict(test_numeric_data)
write_csv(y_pred, 'Prueba1.csv')

# Realizar las predicciones sobre los datos de prueba
y_pred = model.predict(test_numeric_data)

# Escribir las predicciones en un archivo CSV
write_predictions_to_csv(y_pred, 'Prueba1.csv')

In [49]:
# Crear un objeto PCA con 12 componentes y fijar la semilla aleatoria en 42
pca = PCA(n_components=12, random_state=42)

# Transformar los datos de entrada X usando PCA
pca_X = pca.fit_transform(X)

# Generar el modelo de clasificación utilizando los datos transformados
generateModel(pca_X)

TP:  3987
FP:  55
FN:  59
TN:  3457 

[[3987   55]
 [  59 3457]]

Accuracy: 0.9849166446149775
Precision: 0.9843394077448747
Recall:  0.9832195676905574
F1:  0.9837791690381331


KNeighborsClassifier()

In [52]:
# Crear un objeto PCA con 6 componentes y fijar la semilla aleatoria en 42
pca = PCA(n_components=6, random_state=42)
# Transformar los datos de entrada X usando PCA
pca_X = pca.fit_transform(X)
# Generar el modelo de clasificación utilizando los datos transformados
generateModel(pca_X)

TP:  3990
FP:  52
FN:  58
TN:  3458 

[[3990   52]
 [  58 3458]]

Accuracy: 0.9854458851548028
Precision: 0.9851851851851852
Recall:  0.9835039817974972
F1:  0.9843438656419017


KNeighborsClassifier()

In [53]:
# Crear un objeto PCA con 2 componentes y fijar la semilla aleatoria en 42
pca = PCA(n_components=2, random_state=42)
# Transformar los datos de entrada X usando PCA
pca_X = pca.fit_transform(X)
# Generar el modelo de clasificación utilizando los datos transformados
model = generateModel(pca_X)

TP:  3989
FP:  53
FN:  39
TN:  3477 

[[3989   53]
 [  39 3477]]

Accuracy: 0.987827467584017
Precision: 0.984985835694051
Recall:  0.9889078498293515
F1:  0.9869429463525405


In [54]:

# Transformar los datos de prueba usando PCA
pca_test_X = pca.transform(test_numeric_data)

# Realizar las predicciones sobre los datos de prueba transformados
y_pred_pca = model.predict(pca_test_X)

# Escribir las predicciones en un archivo CSV
write_predictions_to_csv(y_pred_pca, 'prueba2.csv')